In [ ]:
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_svd

experiment_name = 'FlattenedGRU'
device = 'cuda:0'
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Модель

In [ ]:
import torch.nn.functional as F


class NeuralClickModel(torch.nn.Module):
    def __init__(self, user_embedding_dim=42, item_embedding_dim=40, num_items=None, num_users=None, 
                 embeddings='explicit', output_dim=1):
        super().__init__()
        self.item_embedding_dim = item_embedding_dim
        self.user_embedding_dim = user_embedding_dim
        self.embeddings = embeddings
        if embeddings == 'neural':
            self.user_embedding = torch.nn.Embedding(num_users, user_embedding_dim)
            self.item_embedding = torch.nn.Embedding(num_items, item_embedding_dim)
        
        self.rnn_layer = torch.nn.GRU(input_size=self.item_embedding_dim, hidden_size=self.user_embedding_dim, batch_first=True)
        self.out_layer = torch.nn.Linear(self.user_embedding_dim, output_dim)


    def forward(self, batch):
        # consider sequential clicks, hence need to flatten slates
        if self.embeddings == 'neural':
            user_embs = self.user_embedding(batch['user_indexes'])[:,0,:].unsqueeze(0).contiguous()
            item_embs = self.item_embedding(batch['slates_item_indexes']).flatten(-3, -2)
        if self.embeddings == 'explicit':
            user_embs = batch['user_embeddings'][:, 0, :].unsqueeze(0).contiguous()
            item_embs = batch['slates_item_embeddings'].flatten(-3, -2)

        shp = batch['slates_item_indexes'].shape
        rnn_out, _ = self.rnn_layer(
            F.dropout(item_embs, p=0.1, training=self.training),
            F.dropout(user_embs, p=0.1, training=self.training)
        )
        # rnn_out = F.dropout1d(rnn_out, p=0.1, training=self.training)
        return self.out_layer(rnn_out).reshape([*shp, 1])


# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [ ]:
d = DummyData()
dummy_loader, dummy_svd = get_dummy_data(d)

for embeddings in ['explicit', 'neural', 'svd']:
    print(f"Evaluating {experiment_name} embeddings")
    model = NeuralClickModel(
        user_embedding_dim=2,
        item_embedding_dim=2,
        output_dim=1, 
        embeddings='neural' if embeddings == 'neural' else 'explicit', 
        num_items = d.n_items,
        num_users = d.n_users
    ).to(device)
    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, encoder=dummy_svd if embeddings == 'svd' else None,
       device=device, lr=1e-3, num_epochs=5000, 
       silent=True
    )


# ContentWise

In [ ]:
content_wise_results = []
c = ContentWise.load('../cw.pkl')
c_train_loader, c_val_loader, c_test_loader, c_svd_encoder = get_train_val_test_svd(c, batch_size=64)
len(c_train_loader), len(c)

In [ ]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        num_items = c.n_items, num_users = c.n_users, 
        user_embedding_dim=c.user_features.shape[-1] if embeddings == 'explicit' else 32, 
        item_embedding_dim=c.item_features.shape[-1] if embeddings == 'explicit' else 32, 
        embeddings='neural' if embeddings == 'neural' else 'explicit', 
        output_dim=1, 
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, encoder=c_svd_encoder if embeddings == 'svd' else None,
       device=device, lr=1e-3, num_epochs=5000, early_stopping=30,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')

# RL4RS

In [ ]:
rl4rs_results = []
r = RL4RS.load('../rl4rs.pkl')
r_train_loader, r_val_loader, r_test_loader, r_svd_encoder = get_train_val_test_svd(r, batch_size=128, )
len(r), len(r_train_loader)

In [ ]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        num_items = r.n_items, num_users = r.n_users, 
        user_embedding_dim=r.user_features.shape[-1] if embeddings == 'explicit' else 32, 
        item_embedding_dim=r.item_features.shape[-1] if embeddings == 'explicit' else 32, 
        embeddings='neural' if embeddings == 'neural' else 'explicit', 
        output_dim=1, 
    ).to(device)
    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, encoder=r_svd_encoder if embeddings == 'svd' else None,
       device='cuda', lr=1e-3, num_epochs=5000, early_stopping=30,
       silent=True
    )
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')